# Data Generating for different purpose using gpt 3.5

In [1]:
import openai
import csv
import os
from tqdm import tqdm

openai.api_key = 'mykey'

def generate_questions(text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Generate one question based on the following text:\n\n{text}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=50
    )

    questions = response.choices[0].message['content'].strip().split('\n')
    questions = [question.lstrip('0123456789. ').strip() for question in questions]
    
    return questions

def extract_context(text, question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Extract sentences from the text that answer the following question:\n\nQuestion: {question}\n\nText:\n{text}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150
    )

    context = response.choices[0].message['content'].strip()
    return context

import time            
def main():
    dataset = []
    directory_path = 'processed/'

    files = os.listdir(directory_path)
    txt_files = [file for file in files if file.endswith('.txt')]
    i = 2
    for file in tqdm(range(i, 347)):
        time.sleep(3)
        file = txt_files[i]
        with open(f'processed/{file}', 'r', encoding='utf-8') as infile:
            text = infile.read()

            questions = generate_questions(text)
            question = questions[0]
            context = extract_context(text, question)
            
            dataset.append({"Question": question, "Context": context})
            
            output_file = f'finetune_dataset/dataset_{i}.csv'
            with open(output_file, mode='w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=["Question", "Context"])
                writer.writeheader()
                writer.writerows(dataset)

    print(f"Dataset saved to {output_file}")

if __name__ == "__main__":
    main()
